In [27]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
from datetime import datetime, timedelta, timezone
import time
import hmac
import re
import glob
from pathlib import Path
pd.options.display.float_format = '{:,.2f}'.format

In [28]:
date_to = '2022-08-09'
period = 180 # days

In [29]:
exchanges = ['kraken','kucoin', 'bitfinex', 'lbank','gemini','bitmart','cryptocom']
pairs = ["ETH/USDT", "ETH/USDC", "ETH/BTC", "ETH/USD", "ETH/EUR", "ETH/DAI","STETH/USDT", "STETH/USDC", "STETH/BTC", "STETH/USD", "STETH/EUR", "STETH/DAI"]

In [30]:
#1 KRAKEN
#https://docs.kraken.com/rest/#tag/Market-Data/operation/getOHLCData

def fetch_kraken_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))
    timestamp_from = timestamp_to - (period-1)*86400
    datetime_to = datetime.fromtimestamp(timestamp_to)  
    datetime_from = datetime.fromtimestamp(timestamp_from)
    
    pair_split = pair.split('/')
    symbol = pair_split[0] + pair_split[1]
    param = {"pair": symbol, "interval": 1440, "since": timestamp_from}
    response = requests.get('https://api.kraken.com/0/public/OHLC', params=param)
    #print(response.text)
    result = json.loads(response.text)
       
    if response.status_code == 200 and result['error'] == []:
        #print (json.dumps(json.loads(response.text), indent=4))
        key = list(result['result'].keys())[0]
        #print(key)
        data = pd.DataFrame(result['result'][key], columns=['timestamp', 'open', 'high', 'low', 'close','vwap','volume', 'count'])
        data['date'] = pd.to_datetime(data['timestamp'], unit='s') 
        data = get_data_formated(data, pair).query('@datetime_from<=index<=@datetime_to')
        data['volume_quote'] = data['volume']*data['close']
        if data is None:
            print("Did not return any data from KRAKEN for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from KRAKEN API for", pair)  
        return pd.DataFrame()

In [31]:
#2 KUCOIN
#https://docs.kucoin.com/#get-klines

def fetch_kucoin_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))+86400
    timestamp_from = timestamp_to - (period)*86400
    
    pair_split = pair.split('/')
    symbol = (pair_split[0] + '-' + pair_split[1])
    param = {"symbol": symbol, "startAt": timestamp_from, "endAt": timestamp_to, "type": "1day", }
    response = requests.get('https://api.kucoin.com/api/v1/market/candles', params=param)
    result = json.loads(response.text)
    
    if response.status_code == 200 and result['code'] == '200000': # and result['data'] is not None
        data = pd.DataFrame(result['data'], columns=['timestamp', 'open', 'close', 'high', 'low', 'volume', 'volume_quote'])
        data['date'] = pd.to_datetime(data['timestamp'], unit='s') 
        data = get_data_formated(data, pair)
        
        if data is None:
            print("Did not return any data from KuCoin for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from KuCoin API for", pair)  
        return pd.DataFrame()
    

In [32]:
#3 BITFINEX
#https://docs.bitfinex.com/reference/rest-public-candles

def fetch_bitfinex_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))+86400
    timestamp_from = timestamp_to - (period)*86400
    pair_split = pair.split('/')
    symbol = 't' + pair_split[0] + pair_split[1]
    param = {"limit": period, "start": timestamp_from * 1000, "end": timestamp_to * 1000}
    path = f'v2/candles/trade:1D:{symbol}/hist'
    response = requests.get(f'https://api-pub.bitfinex.com/{path}', params=param)
    result = json.loads(response.text) 
    
    if response.status_code == 200 and result != []:        
        data = pd.DataFrame(result, columns=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
        data['date'] = pd.to_datetime(data['timestamp'], unit='ms') 
        data = get_data_formated(data, pair)
        data['volume_quote'] = data['volume'] * data['close']
        if data is None:
            print("Did not return any data from BITFINEX for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from BITFINEX API for", pair)  
        return pd.DataFrame()
    

In [33]:
#4 LBANK
#https://www.lbank.info/en-US/docs/index.html#query-k-bar-data

def fetch_lbank_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))+86400
    timestamp_from = timestamp_to - (period-1)*86400
    pair_split = pair.split('/')
    symbol = (pair_split[0] + "_" + pair_split[1]).lower()
    param = {"symbol": symbol, "size": period, "type": "day1", "time": timestamp_from}
    response = requests.get('https://api.lbkex.com/v2/kline.do', params=param)
    result = json.loads(response.text)

    if response.status_code == 200 and result['result'] != 'false':
        data = pd.DataFrame(result['data'], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['timestamp'], unit='s') 
        data['date'] = data['date'].dt.date
        data = get_data_formated(data, pair)
        data['volume_quote'] = data['volume'] * data['close']
        if data is None:
            print("Did not return any data from LBANK for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from LBANK API for", pair)  
        return pd.DataFrame()
    

In [34]:
#5GEMINI
#https://docs.gemini.com/rest-api/#candles

def fetch_gemini_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))
    timestamp_from = timestamp_to - (period-1)*86400
    datetime_to = datetime.fromtimestamp(timestamp_to)  
    datetime_from = datetime.fromtimestamp(timestamp_from)
    pair_split = pair.split('/')
    symbol = pair_split[0] + pair_split[1]
    path = f'/candles/{symbol}/1day'
    response = requests.get(f'https://api.gemini.com/v2'+ path)
  
    if response.status_code == 200:
        data = pd.DataFrame(json.loads(response.text), columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        data["timestamp"] = data["timestamp"]-4*3600*1000 #to strip time
        data['date'] = pd.to_datetime(data['timestamp'], unit='ms') 
        data = get_data_formated(data, pair).query('@datetime_from<=index<=@datetime_to')
        data['volume_quote'] = data['volume']*data['close']
        if data is None:
            print("Did not return any data from GEMINI for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from GEMINI API for", pair)  
        return pd.DataFrame()
 

In [35]:
# 6 BitMart 
#https://developer-pro.bitmart.com/en/spot/#get-k-line

def fetch_bitmart_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))+86400
    timestamp_from = timestamp_to - (period-1)*86400
    pair_split = pair.split('/')
    symbol = (pair_split[0] + '_' + pair_split[1])
    param = {"symbol": symbol, "from": timestamp_from, "to": timestamp_to, "step": "1440"}         
    response = requests.get('https://api-cloud.bitmart.com/spot/v1/symbols/kline', params=param)
    
    if response.status_code == 200:
        data = pd.DataFrame(json.loads(response.text)['data']['klines'], columns=['last_price', 'timestamp', 'volume', 'quote_volume','open', 'close', 'high', 'low'])
        data['date'] = pd.to_datetime(data['timestamp'], unit='s') 
        data.rename(columns = {'quote_volume':'volume_quote'}, inplace = True)
        data = get_data_formated(data, pair)
        if data is None:
            print("Did not return any data from BitMart for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from BitMart API for", pair)  
        return pd.DataFrame()
    

In [36]:
#7 CRYPTOCOM
#https://exchange-docs.crypto.com/spot/index.html#public-get-candlestick

def fetch_cryptocom_daily_data(pair, date_to, period, extended = False):
    timestamp_to = int(datetime.timestamp(datetime.strptime(date_to, '%Y-%m-%d')))
    timestamp_from = timestamp_to - (period-1)*86400
    datetime_to = datetime.fromtimestamp(timestamp_to)  
    datetime_from = datetime.fromtimestamp(timestamp_from)
    
    pair_split = pair.split('/')
    symbol = pair_split[0] + '_' + pair_split[1]

    response = requests.get(f'https://api.crypto.com/v2/public/get-candlestick?instrument_name={symbol}&timeframe=1D')
    #print(response.text)
    #print (json.dumps(json.loads(response.text)['result']['data'], indent=4))
    result = json.loads(response.text)['result']['data']
    
    if response.status_code == 200 and result != None:
        data = pd.DataFrame(result, columns=['t', 'o', 'h', 'l', 'c', 'v'])
        #print(data)
        data['date'] = pd.to_datetime(data['t'], unit='ms')
        data.rename(columns = {'o': 'open', 'h': 'high','l': 'low', 'c': 'close', 'v':'volume'}, inplace = True)
        data = get_data_formated(data, pair).query('@datetime_from<=index<=@datetime_to')
        data['volume_quote'] = data['volume']*data['close']
        if data is None:
            print("Did not return any data from CRYPTO.COM for", pair)
        if extended:
            return data[['open', 'high', 'low', 'close', 'volume', 'volume_quote']]
        else:
            return data[['volume']]
    else:
        print("Did not receieve OK response from CRYPTO.COM API for", pair)  
        return pd.DataFrame()

In [37]:
def get_data_formated(data, pair): 
    data['symbol'] = pair
    data=data.set_index('date')
    for col in ['open','close','high','low','volume','volume_quote']:
        if col in data.columns:
            if type(data[col].values[0]) == str:
                data[col] = data[col].str.replace(',','').astype(np.float64)
    return data

In [38]:
def get_klines_by_exchange_pair(exchange, pair, date_to, period, extended = False):
    if exchange == 'kraken': return fetch_kraken_daily_data(pair, date_to, period, extended)
    elif exchange == 'kucoin': return fetch_kucoin_daily_data(pair, date_to, period, extended)
    elif exchange == 'bitfinex': return fetch_bitfinex_daily_data(pair,date_to, period, extended)
    elif exchange == 'lbank': return fetch_lbank_daily_data(pair,date_to, period, extended)
    elif exchange == 'gemini': return fetch_gemini_daily_data(pair,date_to, period, extended)
    elif exchange == 'bitmart': return fetch_bitmart_daily_data(pair,date_to, period, extended)
    elif exchange == 'cryptocom': return fetch_cryptocom_daily_data(pair,date_to, period, extended)
    else: print('No data for ', exchange)

In [39]:
def get_klines(exchanges, pairs, date_to, period, extended = False):
    klines_by_exchange = {}
    for exchange in exchanges:
        for pair in pairs:
            klines_by_exchange.update({(exchange, pair):get_klines_by_exchange_pair(exchange, pair, date_to, period, extended)})
    return klines_by_exchange
    

In [41]:
klines = get_klines(exchanges, pairs, date_to, period)
def save_klines_dic_as_csv(klines):
    for key in klines.keys():
        klines[key].to_csv(f'{key[0]}_{key[1].split("/")[0]+"_"+key[1].split("/")[1]}.csv', sep=',', encoding='utf-8', index=True)
    print('Done')


Did not receieve OK response from KRAKEN API for STETH/USDT
Did not receieve OK response from KRAKEN API for STETH/USDC
Did not receieve OK response from KRAKEN API for STETH/BTC
Did not receieve OK response from KRAKEN API for STETH/USD
Did not receieve OK response from KRAKEN API for STETH/EUR
Did not receieve OK response from KRAKEN API for STETH/DAI
Did not receieve OK response from KuCoin API for ETH/USD
Did not receieve OK response from KuCoin API for STETH/USDT
Did not receieve OK response from KuCoin API for STETH/USDC
Did not receieve OK response from KuCoin API for STETH/BTC
Did not receieve OK response from KuCoin API for STETH/USD
Did not receieve OK response from KuCoin API for STETH/EUR
Did not receieve OK response from KuCoin API for STETH/DAI
Did not receieve OK response from BITFINEX API for ETH/USDT
Did not receieve OK response from BITFINEX API for ETH/USDC
Did not receieve OK response from BITFINEX API for ETH/DAI
Did not receieve OK response from BITFINEX API for S

In [45]:
save_klines_dic_as_csv(klines)

Done


In [46]:
path = '.' # Path for current folder
csv_files = glob.glob(path + "/*.csv") # Get CSV files list from a folder
def create_list_of_dataframes(csv_files): 
    df_list = [] 
    for file in csv_files:
        file_name_with_no_extension = Path(file).stem
        elements = file_name_with_no_extension.split("_")
        if len(elements) >= 3:
            exchange = elements[0]
            pair = elements[1] + "_" + elements[2]
            df = pd.read_csv(file, index_col = 0) # Read each CSV file into DataFrame
            df['exchange'] = exchange
            df['pair'] = pair
            df_list.append(df)# This creates a list of dataframes
    return df_list
    

In [49]:
df_list = create_list_of_dataframes(csv_files)     
big_df = pd.concat(df_list) # Concatenate all DataFrames

In [50]:
big_df.head()

,exchange,pair,volume
2022-02-11,lbank,ETH_BTC,621.72
2022-02-12,lbank,ETH_BTC,621.09
2022-02-13,lbank,ETH_BTC,633.95
2022-02-14,lbank,ETH_BTC,631.91
2022-02-15,lbank,ETH_BTC,578.88
